## **Kütüphaneleri Yükleme**

In [ ]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from PIL import Image

In [ ]:
tf.config.list_physical_devices("GPU")

## **Veri setini Yükleme ve Kontrol Etme**

In [ ]:
label = []
path = []
fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
for dir_name, _, filenames in os.walk(fish_dir):
    for filename in filenames:
        if os.path.splitext(filename)[-1] == '.png':  # .png uzantılı dosyaları kontrol et
            if dir_name.split('/')[-1] != 'GT':  # GT klasörünü hariç tut
                label.append(os.path.split(dir_name)[-1])  # Klasör adını etiket olarak ekle
                path.append(os.path.join(dir_name, filename))  # Dosya yolunu ekle

data = pd.DataFrame(columns=['path', 'label'])
data['path'] = path
data['label'] = label

In [ ]:
data.columns #Dataframe'in içindeki sütun adlarını çağırır

In [ ]:
data.head() # DataFrame'in ilk beş satırın çağırır

In [ ]:
data.describe()

In [ ]:
# Veri setindeki rastgele 30 veriyi yazdırır
from PIL import Image


sample_data = data.sample(30)  

# Görselleştirme
plt.figure(figsize=(10, 10))
for idx, row in enumerate(sample_data.iterrows()):
    img_path = row[1]['path']
    img = Image.open(img_path)
    
    plt.subplot(5,6, idx+1)
    plt.imshow(img)
    plt.title(row[1]['label'])
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
data.info()

In [ ]:
data["label"].unique()

In [ ]:
# "GT" içermeyen etiketleri seçerek yeni bir veri seti oluştur
data = data[~data['label'].str.contains('GT')]
print("Kalan etiketler:", data['label'].unique())

In [ ]:
def visualize_sample(data, sample_size=30):
    """Veri setinden rastgele örnek görüntüleri görselleştirir."""
    sample_data = data.sample(sample_size)
    
    plt.figure(figsize=(10, 10))
    for idx, row in enumerate(sample_data.iterrows()):
        img_path = row[1]['path']
        img = Image.open(img_path)
        
        plt.subplot(5, 6, idx+1)
        plt.imshow(img)
        plt.title(row[1]['label'])
        plt.axis('off')

    plt.tight_layout()
    plt.show()


visualize_sample(data)

In [ ]:
def plot_class_distribution(data):
    """Sınıf dağılımını pasta grafiği ile görselleştirir."""
    class_counts = data['label'].value_counts()
    class_counts.plot(kind='pie', figsize=(8, 8), autopct='%1.1f%%')

    plt.title('Sınıf Dağılımı')
    plt.ylabel('')  
    plt.show()


plot_class_distribution(data)


In [ ]:
data['label'].value_counts()

# Train - Test Split 

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

# Eğitim veri seti
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='path',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    subset='training'
)

# Doğrulama veri seti
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='path',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    subset='validation'
)

# Test veri seti
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    x_col='path',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

In [ ]:
# Eğitim veri setinin büyüklüğünü yazdırma
print("Eğitim veri setinin büyüklüğü:", train_generator.samples)
print("Test veri setinin büyüklüğü:", test_generator.samples)
print("Doğrulama veri setinin büyüklüğü:", val_generator.samples)

In [ ]:
train_images, train_labels = next(train_generator)

# Alınan görüntülerin ve etiketlerin şekillerini yazdırma
print("Eğitim görüntülerinin şekli:", train_images.shape) 
print("Eğitim etiketlerinin şekli:", train_labels.shape)    

In [ ]:
images, labels = next(test_generator)

# Alınan görüntülerin ve etiketlerin şekillerini yazdırma
print("Test görüntülerin şekli:", images.shape) 
print("Test etiketlerin şekli:", labels.shape)    


In [ ]:
images, labels  = next (val_generator)
print("Validasyon görüntülerin şekli:", images.shape)  
print("Validasyon etiketlerin şekli:", labels.shape) 

# Model Oluşturma

In [2]:
from tensorflow.keras import layers, models
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
# Konvolüsyonel katmanlar
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

* Konvolüsyonel katmanlar görüntülerin özelliklerini çıkarır. Bu, özellikle görüntü tabanlı verilerde kullanılır. 2D konvolüsyonel katmanlar (Conv2D), görüntünün kenarlarını, köşelerini ve diğer önemli özelliklerini tespit etmek için filtreler uygular
* Max pooling katmanı, görüntünün boyutunu küçültmek (downsampling) için kullanılır. Bu işlem, modelin daha az hesaplama gücüyle daha hızlı çalışmasına yardımcı olur.


In [ ]:
model.add(tf.keras.layers.Flatten())

* model.add(tf.keras.layers.Flatten()) ifadesi, konvolüsyonel katmanlardan elde edilen çok boyutlu (3D) verileri düzleştirmek için kullanılır. 

In [ ]:
# Tam bağlantılı katmanlar
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dropout(0.4))  # Dropout overfitting'i azaltabilir

model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dropout(0.3))

* Dense katmanlar, öğrenme kapasitesini artırırken, Dropout katmanları aşırı öğrenmeyi önleyerek modelin daha iyi performans göstermesini sağlar.

In [ ]:
model.add(tf.keras.layers.Dense(9, activation="softmax"))

* Model, girdi verilerini alıp her bir sınıf için olasılık tahminleri yapar. Örneğin, model bir görüntüyü analiz ettiğinde, her bir sınıf için (1. sınıf, 2. sınıf, ..., 9. sınıf) olasılıklar verir ve en yüksek olasılığa sahip olan sınıf tahmin edilen sınıf olarak kabul edilir.

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# EarlyStopping tanımlama
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

* EarlyStopping, modelin eğitimini otomatik olarak durdurarak, aşırı öğrenmeyi engeller ve en iyi performansı sağlayan ağırlıkların kullanılmasını sağlar. Bu sayede, gereksiz hesaplama süresinden tasarruf edilir ve modelin genel performansı artırılır.

In [ ]:
# Modeli eğitme
results = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,  # Daha fazla epoch deneyebilirsiniz
    callbacks=[early_stopping]
)

* Model, eğitim verisi üzerinde iyi bir performans sergileyerek doğruluğu artırdı, ancak bazı epoch'larda doğrulama verisi üzerindeki performans dalgalanmaları görüldü. Bu, aşırı öğrenme (overfitting) belirtisi olabilir. Early Stopping mekanizması, aşırı öğrenmeyi önlemek için kullanıldı ve modelin en iyi performans gösterdiği epoch'taki ağırlıkları geri yüklenerek en iyi sonuçlar elde edilmeye çalışıldı.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns

# 1. Eğitim ve doğrulama kayıp ve doğruluk grafiğini çizme
def plot_training_history(history):
    acc = results.history['accuracy']
    val_acc = results.history['val_accuracy']
    loss = results.history['loss']
    val_loss = results.history['val_loss']
    epochs = range(1, len(acc) + 1)

    # Accuracy grafiği
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    # Loss grafiği
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

# 2. Confusion Matrix ve Classification Report
def evaluate_model(model, test_generator):
    # Test setine modelin tahminlerini uygulama
    Y_pred = model.predict(test_generator)
    y_pred = np.argmax(Y_pred, axis=1)

    # Gerçek etiketler
    y_true = test_generator.classes

    # Confusion matrix oluşturma
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, 
                xticklabels=list(test_generator.class_indices.keys()), 
                yticklabels=list(test_generator.class_indices.keys()))
    plt.title('Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

    # Classification report oluşturma
    print('Classification Report')
    target_names = list(test_generator.class_indices.keys())
    print(classification_report(y_true, y_pred, target_names=target_names))

plot_training_history(results)  

evaluate_model(model, test_generator)  


In [ ]:
import tensorflow as tf
from tensorflow import keras
from kerastuner import HyperModel, RandomSearch
from tensorflow.keras.callbacks import EarlyStopping


def build_model(hp):
    model = keras.models.Sequential()
    
    # Flatten katmanı
    model.add(keras.layers.Flatten(input_shape=(224, 224, 3)))
    
    # İlk dense katmanı (nöron sayısı için arama)
    model.add(keras.layers.Dense(
        units=hp.Int('units1', min_value=128, max_value=512, step=128), 
        activation='relu'))
    
    # Dropout oranı (ilk dropout katmanı)
    model.add(keras.layers.Dropout(
        rate=hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)))
    
    # İkinci dense katmanı
    model.add(keras.layers.Dense(
        units=hp.Int('units2', min_value=128, max_value=512, step=128), 
        activation='relu'))
    
    # Dropout oranı (ikinci dropout katmanı)
    model.add(keras.layers.Dropout(
        rate=hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)))
    
    # Çıkış katmanı
    model.add(keras.layers.Dense(9, activation='softmax'))

    # Öğrenme oranı için optimizer yapılandırması
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Hiperparametre arama fonksiyonunu ayarlayalım
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  
    executions_per_trial=2, 
    directory='hyperparam_search',
    project_name='cnn_tuning')


tuner.search(train_generator, validation_data=val_generator, epochs=10, callbacks=[early_stopping])

# En iyi modeli alalım
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# En iyi modelle eğitimi başlatalım
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_generator, validation_data=val_generator, epochs=15, callbacks=[early_stopping])

# En iyi hiperparametreler
print(f"Optimal units1: {best_hps.get('units1')}")
print(f"Optimal units2: {best_hps.get('units2')}")
print(f"Optimal dropout1: {best_hps.get('dropout1')}")
print(f"Optimal dropout2: {best_hps.get('dropout2')}")
print(f"Optimal learning rate: {best_hps.get('learning_rate')}")
